In [1]:
#导入必须库
import json
import os

In [2]:
#打开提取的题目json
with open('aime_amc_new.jsonl', 'r') as f:
    data = [json.loads(l) for l in f.readlines()]

In [ ]:
#以下是gpt构造器
import openai
from openai import AzureOpenAI
import json
clientT = AzureOpenAI(
    
    api_version="2023-05-15"
)

save_pth='claude_result/'

class gpt_interface:
    def __init__(self):
        
        self.history = []
        self.history2 = {}
    def get_response2(self,question,id):
        if question in self.history2:
            return self.history2[question],id
        #不在，则调用gpt
        response = clientT.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": "Please generate a natural language proof of the following question. Make it as detailed as possible."
            },
            {
                "role": "user",
                "content": question
            }
        ]
    )
        result_str=(response.choices[0].message.content)
        self.history2[question]=result_str
        return result_str,id
    
    def get_response(self,question,answer,id):
        #判断question是否在history中,如果在则直接
        for item in self.history:
            if item['question']==question:
                return item
        response = self.client.chat.completions.create(
            # model="o1-mini-all",
            # model = "gpt-3.5-turbo",
            model = ["o1-preview-all","claude-3-5-sonnet-20241022"][1],
            messages=[
                {"role": "system", "content": "You are a Lean4 expert."},
                {"role": "user", "content":
                '''


'''%(question,answer)
                },
            ],
            stream=True,
            timeout=1000
            )
        result_str=''
        extra_str=''
        for chunk in response:
            # print(chunk.__dict__)
            if chunk.choices[0].delta.content != None:
              result_str+=(chunk.choices[0].delta.content)
              # print(chunk.choices[0].delta.content,end='')
            # 检测是否已经完成输出
            out = re.search(r'```lean\n(.*?)\n```', result_str, re.DOTALL)
            if out:
               extra_str = out.group(1)
               break
        self.history.append({'question':question,'answer':answer,'response':extra_str,'original':result_str})
        return {'question':question,'answer':answer,'response':extra_str,'original':result_str},id
    def save(self):
      with open(save_pth+'history_claude.json','w') as f:
          json.dump(self.history,f)

In [24]:
#以下是多线程构造器
import concurrent.futures
def parallel_requests(func,prompts_list,max_process=16):
    results = [0]*len(prompts_list)
    # 使用线程池来并行处理请求
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_process) as executor:
        # 提交所有的请求到线程池
        futures = [executor.submit(func, prompt,id) for id,prompt in enumerate(prompts_list)]
        # 收集所有请求的结果
        for future in concurrent.futures.as_completed(futures):
            try:
                result, id = future.result()
                results[id]=result
            except Exception as exc:
                results.append({"error": str(exc)})

    return results

In [10]:
test1000=gpt_interface()
#产生一个gpt对象

In [ ]:
#构造题目提问序列
prompts=[]
for item in data:
    prompts.append(item['informal_prefix'])

'For real numbers $a$ and $b$, define $a$$$b = (a - b)^2$. What is $(x - y)^2$$$(y - x)^2$?\n$\\textbf{(A)}\\ 0 \\qquad \\textbf{(B)}\\ x^2 + y^2 \\qquad \\textbf{(C)}\\ 2x^2 \\qquad \\textbf{(D)}\\ 2y^2 \\qquad \\textbf{(E)}\\ 4xy$'

In [40]:
prompt_list=parallel_requests(test1000.get_response2,prompts,64)

['To solve this problem, we begin by interpreting the operation defined in the problem statement. The operation between two real numbers \\(a\\) and \\(b\\), denoted \\(a$$$b\\), is defined as:\n\n\\[\na$$$b = (a - b)^2\n\\]\n\nWe need to calculate \\((x-y)^2$$$(y-x)^2\\). To apply the operation, we apply the definition:\n\nLet \\(a = (x-y)^2\\) and \\(b = (y-x)^2\\).\n\nAccording to the definition provided:\n\n\\[\na - b = (x-y)^2 - (y-x)^2\n\\]\n\nWe need to simplify the expression for \\(a - b\\). Begin by expanding the squares:\n\n\\[\n(x-y)^2 = x^2 - 2xy + y^2\n\\]\n\\[\n(y-x)^2 = y^2 - 2yx + x^2 = x^2 - 2xy + y^2\n\\]\n\nThus, both \\((x-y)^2\\) and \\((y-x)^2\\) simplify to the same expression:\n\n\\[\n(x-y)^2 = x^2 - 2xy + y^2\n\\]\n\\[\n(y-x)^2 = x^2 - 2xy + y^2\n\\]\n\nSo, the difference is:\n\n\\[\n(x-y)^2 - (y-x)^2 = (x^2 - 2xy + y^2) - (x^2 - 2xy + y^2) = 0\n\\]\n\nAs a result, using the operation, we find \\(a - b = 0\\).\n\nNow apply the definition of the operation:\n\n\

In [9]:
#回写题目
for i in range(len(data)):
    data[i]['formal_statement']+='\n/-%s-/\n'%hints[i]
with open('aime_amc_orihint.jsonl', 'w') as f:
    for item in data:
        f.write(json.dumps(item)+'\n')

In [ ]:
with open("temp1.txt",'r') as f:
    hints=f.readlines()
    hints=[x[2:-3] for x in hints]

To solve this problem, we need to determine how many bricks are in the chimney based on the work rates of Brenda and Brandon, as well as the effect of their decreased efficiency when working together.\n\nFirst, let's calculate the individual work rates of Brenda and Brandon. \n\n- **Brenda's work rate:**\n  If Brenda can build the chimney alone in 9 hours, her work rate is:\n  \\[\n  \\frac{1}{9} \\text{ of the chimney per hour}\n  \\]\n\n- **Brandon's work rate:**\n  If Brandon can build the chimney alone in 10 hours, his work rate is:\n  \\[\n  \\frac{1}{10} \\text{ of the chimney per hour}\n  \\]\n\nIf there were no decreased efficiency, their combined work rate would be:\n\\[\n\\frac{1}{9} + \\frac{1}{10} = \\frac{10}{90} + \\frac{9}{90} = \\frac{19}{90} \\text{ of the chimney per hour}\n\\]\n\nHowever, working together, they talk a lot, decreasing their efficiency by 10 bricks per hour. They complete the chimney in 5 hours, which suggests their collective work rate is:\n\\[\n\\fra